## Optimizer: Finding best hyper-parameters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install "numpy<1.24.0"
!pip install transformers datasets scikit-optimize
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from transformers import BartTokenizer, BartForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path

from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import numpy as np
from skopt import gp_minimize

In [ ]:
#Define the space of hyperparameters to search
search_space = [
  Real(1e-5, 5e-5, name='learning_rate'),
  Real(0.01, 0.2,  name='weight_decay'),
  Integer(3, 5, name='num_train_epochs'),
  Integer(8, 32, name='per_device_train_batch_size', prior='log-uniform'),
  Integer(8, 64, name='per_device_eval_batch_size', prior='log-uniform'),
]


@use_named_args(search_space)
def objective(learning_rate, num_train_epochs, per_device_train_batch_size, per_device_eval_batch_size, weight_decay):
    __file__ = "/content/drive/MyDrive/FinalProject/Models/BART/BART_optimizer.ipynb"
    file_name = "dataset_balanced_4000"
    ext ="xlsx"
    path_type = "Balanced"

    current_file_path = Path(__file__).parent
    path_to_project = current_file_path.parents[1]

    df = pd.read_excel(f"{path_to_project}/Data/Datasets/{path_type}/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/BART/Output/{path_type}/{file_name}_optimizer"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    # Select the text and label columns
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)
    X = df['review'].values
    y = df['label'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


    # Encode the labels to a numeric format
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Initialize the tokenizer for RoBERTa
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
    # Tokenization function
    def tokenize_function(texts):
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)


    # Tokenize the data
    train_encodings = tokenize_function(X_train.tolist())
    val_encodings = tokenize_function(X_test.tolist())

    # Create dataset objects
    train_dataset = ReviewDataset(train_encodings, y_train_encoded)
    val_dataset = ReviewDataset(val_encodings, y_test_encoded)

    # Initialize the model for each fold
    model = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels=len(label_encoder.classes_))


    # Define training arguments for each fold, adjust hyperparameters as needed
    training_args = TrainingArguments(
        output_dir=f"{dump_dir}/res",
        num_train_epochs= int(num_train_epochs),
        per_device_train_batch_size= int(per_device_train_batch_size),
        per_device_eval_batch_size= int(per_device_eval_batch_size),
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir=f"{dump_dir}/logs",
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        max_grad_norm=1.0,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_strategy="epoch",
        save_total_limit=2,
        lr_scheduler_type='linear'
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
        )

    # Train
    trainer.train()
    # Evaluate
    results = trainer.evaluate()
    neg_accuracy = -results['eval_accuracy']

    print(results['eval_accuracy'])
    print(neg_accuracy)

    shutil.rmtree(dump_dir)
    return neg_accuracy

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # If logits are wrapped in a tuple, unwrap them. Otherwise, leave as is.
    if isinstance(logits, tuple):  # Adjust this line if the structure is different
        logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(predictions, labels)}


# Custom dataset class
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

result = gp_minimize(objective, search_space, n_calls=20, random_state=0)

print("Best parameters:")
print(result.x)




Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.315300,1.025729,0.616250
2,0.840500,0.735230,0.708750
3,0.651700,0.691405,0.723750
4,0.527800,0.710679,0.737500
5,0.359000,0.695500,0.766250


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

0.76625
-0.76625


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

0.77875
-0.77875


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.061700,0.840064,0.703750
2,0.689000,0.683015,0.747500
3,0.453000,0.771757,0.780000
4,0.308600,0.892963,0.776250
5,0.183600,1.057932,0.780000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

0.78
-0.78
Best parameters:
[3.328079168300429e-05, 0.11210091359531205, 5, 9, 21]
